In [1]:
import os
import json
import pandas as pd
import numpy as np

current_directory = os.getcwd()
path_components = current_directory.split(os.path.sep)

results_dir = os.path.sep.join(path_components[:-2])
results_dir = os.path.join(results_dir,'.results')
print(results_dir)

c:\Users\achom\Desktop\Main\Code\Scientific-Computing\Physics-Neural-Networks\PINN\.results


In [12]:
def find_largest_iteration(folder_path):
    subdirectories = [d for d in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, d))]
    iteration_numbers = [int(d.split('_')[1]) for d in subdirectories if d.startswith('iter_')]
    if not iteration_numbers:
        return None
    largest_iteration_folder = os.path.join(folder_path, f'iter_{max(iteration_numbers)}')
    return largest_iteration_folder

def find_loss_csv(folder_path):
    largest_iteration_folder = find_largest_iteration(folder_path)
    if largest_iteration_folder:
        loss_csv_path = os.path.join(largest_iteration_folder, 'loss.csv')
        if os.path.isfile(loss_csv_path):
            return loss_csv_path
    return None

def get_last_iteration_losses(loss_csv_path):
    df = pd.read_csv(loss_csv_path)
    last_row = df.iloc[-1]
    losses = last_row[list(df.columns)]
    return losses

def read_results_json(folder_path):
    results_json_path = os.path.join(folder_path, 'results_values.json')
    if os.path.isfile(results_json_path):
        with open(results_json_path, 'r') as json_file:
            results_data = json.load(json_file)
        return results_data
    else:
        return None

def get_results_by_sim(folder_path):
    loss_csv_path = find_loss_csv(folder_path)
    if loss_csv_path:
        results = get_last_iteration_losses(loss_csv_path)
        results_data = read_results_json(folder_path)
        if results_data:
            for name in results_data:
                results[name] = float(results_data[name])
            return results
    return None

In [13]:
folders = ['SIMS_1','SIMS_2','SIMS_3']
data = dict()
for folder in folders:
    folder_path = os.path.join(results_dir,folder)
    sims = os.listdir(folder_path)
    for sim in sims:
        sim_path = os.path.join(folder_path,sim)
        result = get_results_by_sim(sim_path)
        data[sim] = result

data_filtered = {sim: result for sim, result in data.items() if result is not None}
df = pd.DataFrame.from_dict(data_filtered, orient='index')

filter_list = ['Gsolv_value','L2_analytic','Iu','Id','Loss_NN1', 'Loss_NN2']
df_filtered = df.filter(filter_list)

excel_file_path = 'results_temp.xlsx'
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name='Complete', index=True)
    df_filtered.to_excel(writer, sheet_name='Filtered', index=True)

In [14]:
df_filtered

,Gsolv_value,L2_analytic,Iu,Id,Loss_NN1,Loss_NN2
S1,232.195,197.50,5.272786e-06,3.213459e-03,4.789000e-01,3.303000e-03
S10,-248.181,512.60,4.554390e-04,2.478091e-04,3.277000e+00,7.797000e-04
S11,32.485,1161.00,1.618778e-03,7.103438e-03,1.097000e+05,8.831000e-03
S12,0.712,25.21,7.636343e-08,5.979953e-07,3.801000e-01,5.465000e+02
S13,30.944,723.60,5.083430e-04,1.863480e-02,2.616000e-02,7.106000e+02
S14,48.058,1875.00,3.844414e-03,4.607599e-02,6.232000e+04,2.366000e+02
S15,-1.187,44.21,4.288733e-08,2.057204e-08,1.817000e-04,2.251000e-04
S16,4.942,306.40,1.168818e-04,6.287569e-04,1.176000e-02,2.104000e+02
S17,-14.575,282.70,3.194469e-05,6.966335e-06,2.549000e+00,1.696000e+01
S18,97.865,2269.00,4.602050e-03,4.087346e-02,5.714000e+01,2.354000e+02
